# Classification of arrythmias

Explanation of using interpretable algorithms in healthcare to be able to detect features that explain why the arrythmia exists

In [ ]:
# Start with XGBOOST, then SVM wth a couple of kernels, then Naive Bayes, compare metrics, specially important false negatives and F1